In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
%matplotlib inline
import csv

from functools import reduce

# 1. compare based on doi

## import data

In [2]:
wos_doi = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')[['UT (Unique WOS ID)','DOI']]
wos_doi.head()

,UT (Unique WOS ID),DOI
0,WOS:A1968A855300005,10.1055/s-0028-1105102
1,WOS:A1968A855300004,10.1055/s-0028-1105101
2,WOS:A1968A855300003,10.1055/s-0028-1105100
3,WOS:A1968B963600036,NaN
4,WOS:A1968B963600037,NaN


In [3]:
wos_doi.count().to_frame()

,0
UT (Unique WOS ID),932
DOI,875


In [4]:
wos_doi.columns

Index(['UT (Unique WOS ID)', 'DOI'], dtype='object')

In [5]:
# wos_doi.drop_duplicates(subset=['UT (Unique WOS ID)'])

In [6]:
## How many studies in WOS ?
wos_doi.drop_duplicates(inplace = True)
print(">>>", " total number of WOS publications : ", wos_doi.shape[0])

>>>  total number of WOS publications :  932


In [7]:
## How many WOS studies have DOI  ?
wos_have_doi_have_doi = wos_doi[wos_doi['DOI'].notnull()]
print(">>>", " total number of WOS publications with DOI : ", wos_have_doi_have_doi.shape[0])

>>>  total number of WOS publications with DOI :  875


In [8]:
## How many WOS studies have DOI  ?
wos_have_doi_no_doi = wos_doi[wos_doi['DOI'].isnull()]
print(">>>", " total number of WOS publications with no DOI : ", wos_have_doi_no_doi.shape[0])

>>>  total number of WOS publications with no DOI :  57


In [9]:
wos_have_doi_no_doi.count()

UT (Unique WOS ID)    57
DOI                    0
dtype: int64

## get Unpaywall data via the API

In [10]:
import os
myemail = 'dmbogning15@gmail.com'

In [11]:
import requests
import json
import time

def get_doi_data(doi):
    email = myemail or ''
    url = 'https://api.unpaywall.org/v2/{}?email={}'.format(doi,email)
    r = requests.get(url)
    data = r.json()
    response_time = r.elapsed.total_seconds()
    # simple rule for sleeping if responses are slow
    if response_time > 5.0:
        print(response_time, 'seconds for last request')
        print('Unpaywall slow to respond. Sleep for a few seconds.')
        time.sleep(int(response_time))
        print('Finished sleeping')
    return data

In [12]:
wos_doi_na = wos_doi.dropna()
list_wos_doi = list(wos_doi_na['DOI'])
list_wos_doi[:5]

['10.1055/s-0028-1105102',
 '10.1055/s-0028-1105101',
 '10.1055/s-0028-1105100',
 '10.1055/s-0028-1105098',
 '10.1055/s-0028-1105103']

In [13]:
len(list_wos_doi)

875

In [14]:
# Python code to split a list
# into sublists of given length.
#list_doi_all = list(databases_merged.reset_index()['DOI'])#[:26]
list_wos_doi_split = [list_wos_doi[x:x+1000] for x in range(0, len(list_wos_doi), 1000)]

In [15]:
len(list_wos_doi)

875

In [16]:
range(len(list_wos_doi_split))

range(0, 1)

In [17]:
# list_wos_doi_split[0]

In [18]:
# data_upw = pd.DataFrame()
for elt in range(len(list_wos_doi_split)): # len(list_doi_split)  # range(21, 25)
    data_upw = []
    for doi in list_wos_doi_split[elt] :
        data_doi = get_doi_data(doi)
#         data_upw = pd.concat([data_upw,data_doi])
        data_upw.append(data_doi)
    
    data_upw = pd.json_normalize(data_upw)
    
    data_upw.to_csv(f'../DATA_PREPROCESSING/data_upw.csv')
    data_upw.to_excel(f'../DATA_PREPROCESSING/data_upw.xls')
    data_upw.to_pickle(f'../DATA_PREPROCESSING/data_upw.pkl')
    print(f"#+++>>> total number of publication found in unpaywall for list_wos_doi_split_{elt} :", data_upw.shape[0]," #") 

C:\Users\dmbog\AppData\Local\Temp\ipykernel_11604\151552576.py:12: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  data_upw.to_excel(f'../TABLES_gender/data_upw.xls')


#+++>>> total number of publication found in unpaywall for list_wos_doi_split_0 : 875  #


In [65]:
merged_df = pd.merge(wos_doi.reset_index(), data_upw.rename({'doi':'DOI'}, axis=1), on='DOI', how = 'left').drop('index', axis=1)
merged_df = merged_df.drop_duplicates(['UT (Unique WOS ID)'])

merged_df.to_csv(f'../DATA_PREPROCESSING/marburg_data_upw.csv')
merged_df.to_excel(f'../DATA_PREPROCESSING/marburg_data_upw.xls')
merged_df.to_pickle(f'../DATA_PREPROCESSING/marburg_data_upw.pkl')
merged_df

C:\Users\dmbog\AppData\Local\Temp\ipykernel_11604\2456938019.py:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  merged_df.to_excel(f'../DATA_PREPROCESSING/marburg_data_upw.xls')


,UT (Unique WOS ID),DOI,doi_url,title,genre,is_paratext,published_date,year,journal_name,journal_issns,...,first_oa_location.version,first_oa_location.host_type,first_oa_location.is_best,first_oa_location.pmh_id,first_oa_location.endpoint_id,first_oa_location.repository_institution,first_oa_location.oa_date,HTTP_status_code,error,message
0,WOS:A1968A855300005,10.1055/s-0028-1105102,https://doi.org/10.1055/s-0028-1105102,Zur Neuropathologie der durch grüne Meerkatzen...,journal-article,False,1968-03-01,1968.0,DMW - Deutsche Medizinische Wochenschrift,"0012-0472,1439-4413",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:A1968A855300004,10.1055/s-0028-1105101,https://doi.org/10.1055/s-0028-1105101,Die pathologische Anatomie der „Marburg-Virus”...,journal-article,False,1968-03-01,1968.0,DMW - Deutsche Medizinische Wochenschrift,"0012-0472,1439-4413",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WOS:A1968A855300003,10.1055/s-0028-1105100,https://doi.org/10.1055/s-0028-1105100,Zur Epidemiologie der Erkrankung von Menschen ...,journal-article,False,1968-03-01,1968.0,DMW - Deutsche Medizinische Wochenschrift,"0012-0472,1439-4413",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WOS:A1968B963600036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404.0,True,'10.1093/jambio/lxac067' isn't in Unpaywall. S...
5,WOS:A1968B963600037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404.0,True,'10.1093/jambio/lxac067' isn't in Unpaywall. S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,WOS:000398598700003,10.1098/rstb.2016.0294,https://doi.org/10.1098/rstb.2016.0294,Offering patients more: how the West Africa Eb...,journal-article,False,2017-04-10,2017.0,Philosophical Transactions of the Royal Societ...,"0962-8436,1471-2970",...,publishedVersion,publisher,True,None,None,None,2017-04-10,NaN,NaN,NaN
985,WOS:000309742800063,10.1371/journal.pone.0045479,https://doi.org/10.1371/journal.pone.0045479,Virological and Serological Findings in Rouset...,journal-article,False,2012-09-17,2012.0,PLoS ONE,1932-6203,...,publishedVersion,publisher,True,None,None,None,2012-09-17,NaN,NaN,NaN
986,WOS:000460640400002,10.1093/infdis/jiy251,https://doi.org/10.1093/infdis/jiy251,Suspected Exposure to Filoviruses Among People...,journal-article,False,2018-06-18,2018.0,The Journal of Infectious Diseases,"0022-1899,1537-6613",...,publishedVersion,repository,True,oai:europepmc.org:X33rPWYHCQKFfY3qZgSP,b5e840539009389b1a6,PubMed Central - Europe PMC,None,NaN,NaN,NaN
987,WOS:000460640400054,10.1093/infdis/jiy435,https://doi.org/10.1093/infdis/jiy435,Infection Rates and Risk Factors for Infection...,journal-article,False,2018-09-07,2018.0,The Journal of Infectious Diseases,"0022-1899,1537-6613",...,publishedVersion,publisher,True,None,None,None,2018-09-07,NaN,NaN,NaN


In [44]:
len(wos_doi['DOI'].unique())

876

In [42]:
len(merged_df['DOI'].unique())#.count()

876

In [66]:
merged_df['DOI'].count()

875

In [26]:
data_upw.columns

Index(['doi', 'doi_url', 'title', 'genre', 'is_paratext', 'published_date',
       'year', 'journal_name', 'journal_issns', 'journal_issn_l',
       'journal_is_oa', 'journal_is_in_doaj', 'publisher', 'is_oa',
       'oa_status', 'has_repository_copy', 'best_oa_location',
       'first_oa_location', 'oa_locations', 'oa_locations_embargoed',
       'updated', 'data_standard', 'z_authors', 'best_oa_location.updated',
       'best_oa_location.url', 'best_oa_location.url_for_pdf',
       'best_oa_location.url_for_landing_page', 'best_oa_location.evidence',
       'best_oa_location.license', 'best_oa_location.version',
       'best_oa_location.host_type', 'best_oa_location.is_best',
       'best_oa_location.pmh_id', 'best_oa_location.endpoint_id',
       'best_oa_location.repository_institution', 'best_oa_location.oa_date',
       'first_oa_location.updated', 'first_oa_location.url',
       'first_oa_location.url_for_pdf',
       'first_oa_location.url_for_landing_page', 'first_oa_location.

In [21]:
data_upw['is_oa'].value_counts()

True     645
False    228
Name: is_oa, dtype: int64

In [27]:
data_upw['oa_status'].value_counts()

gold      252
closed    228
green     187
bronze    152
hybrid     54
Name: oa_status, dtype: int64